In [47]:
import os, sys, pickle

import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline 
import os

import jp_mpl as jplot

from scipy import optimize
from matplotlib.colors import LogNorm
import pickle

# My tools
import jp_analysis as analysis
import reflection_fit_v2 as reflection_fit
reload(reflection_fit)

from copy import deepcopy

## This step uses the histograms and calculates the peak positions of the first peak when the fit is initialized. The second peak is calculated as part of the minimization.

In [48]:
# Load PMT positions
pmt_info = pickle.load(open('/home/jpyanez/snoplus/snoplus_python/pmt_positions.pckl'))


In [49]:
# Settings
#myinput = '/home/jpyanez/scratch/laserball/337/p300_n300_0'
histdir = '/sb/project/qbs-015-ac/jpyanez/data/SOC_histograms_python'
fitdir  = '/sb/project/qbs-015-ac/jpyanez/data/SOC_reflection_fits'


In [50]:
usable_run_list =  [17375,
100556,
100558,
101427,
101428,
101432,
101433,
102518,
102529,
102552,
102554,
102570,
102572,
102574]

In [51]:
original_infile_list = os.listdir(histdir)
socfiles = []
for one_file in original_infile_list:
    for usable_run in usable_run_list:
        if ("%i" % usable_run in one_file) and ('pckl' in one_file):
            socfiles.append(one_file)
            break
            
print 'List of good run files'
socfiles

List of good run files


['SOC_0000101432.pckl',
 'SOC_0000101428.pckl',
 'SOC_0000101427.pckl',
 'SOC_0000100558.pckl',
 'SOC_0000017375.pckl',
 'SOC_0000100556.pckl',
 'SOC_0000101433.pckl',
 'SOC_0000102518.pckl',
 'SOC_0000102529.pckl',
 'SOC_0000102552.pckl',
 'SOC_0000102554.pckl',
 'SOC_0000102570.pckl',
 'SOC_0000102572.pckl',
 'SOC_0000102574.pckl']

In [52]:
use_file = 4
data = pickle.load(open(os.path.join(histdir, socfiles[use_file])))

In [53]:
non_bottom = (pmt_info['xyz'][:,2]>-6000)#*(pmt_info['xyz'][:,2]<6000)


In [54]:
peakfit = reflection_fit.FitLBpos( data = data,
                                   pmt_xyz = pmt_info['xyz'],
                                   fix_xy = True,
                                   pmtbool = non_bottom,
                                   print_call = True)
my_x0 = np.concatenate((data['manip_pos']+np.random.normal(0., 20., size=3), [1.35]))
print 'Difference wrt MANIP', data['manip_pos'] - my_x0[:3]

Calculating first peak position
Difference wrt MANIP [ -7.62116241  12.91597766  -8.47322699]


In [55]:
peakfit.x = data['manip_pos'][0]
peakfit.y = data['manip_pos'][1]

In [57]:
wrapfcn = lambda p: peakfit(*p)

In [58]:
peakfit.x = data['manip_pos'][0]
peakfit.y = data['manip_pos'][1]
print peakfit.x, peakfit.y
peakr = optimize.minimize(wrapfcn,
                          x0 = my_x0[2:],
                          method = 'SLSQP',
                          bounds=((-6500,6500),
                                  (0.5, 2.)),
                          options={'ftol':1E-7, 'maxiter':1000})

0.0 -254.0
Calculating the second peak position
FCN 		 u 	 v 	 w 	 n 	 nbool
24300.4458467 	0.0 	-254.0 	33.4732269874 	1.35 	7144
24300.4458467 	0.0 	-254.0 	33.4732269874 	1.35 	7144
24300.445848 	0.0 	-254.0 	33.4732270023 	1.35 	7144
24300.4827793 	0.0 	-254.0 	33.4732269874 	1.3500000149 	7144
68041911.1841 	0.0 	-254.0 	33.4610743454 	0.499999999574 	7144
514871.177015 	0.0 	-254.0 	33.4720117232 	1.26499999996 	7144
8477.45800409 	0.0 	-254.0 	33.4730444379 	1.33723182335 	7144
8477.45800409 	0.0 	-254.0 	33.4730444379 	1.33723182335 	7144
8477.45800455 	0.0 	-254.0 	33.4730444528 	1.33723182335 	7144
8477.45800409 	0.0 	-254.0 	33.4730444379 	1.33723183825 	7144
8477.32476932 	0.0 	-254.0 	33.4687252608 	1.33723191896 	7144
8477.32476932 	0.0 	-254.0 	33.4687252608 	1.33723191896 	7144
8477.32476978 	0.0 	-254.0 	33.4687252757 	1.33723191896 	7144
8477.32476932 	0.0 	-254.0 	33.4687252608 	1.33723193386 	7144
8476.65903042 	0.0 	-254.0 	33.4471331729 	1.33723240197 	7144
8476.6

In [10]:
bestfit_value = 9E9
results = errors = None
all_results = {}
peakfit.second_peak_error = []
    
print '\n****************Iteration ', iFit, '********************'
wrapfcn = lambda p: peakfit(*p)

peakr = optimize.minimize(wrapfcn,
                          x0 = my_x0,
                          method = 'SLSQP',
                          bounds=((-6500,6500),
                            (-6500,6500),
                            (-6500,6500),
                            (0.5, 2.)),
                    options={'ftol':1E-7, 'maxiter':1000})

all_results[iFit] = {'fit':deepcopy(peakr),
                     'npmts':np.sum(peakfit.thisbool)}


Calculating first peak position


KeyboardInterrupt: 